In [0]:
# Importing required libraries
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import re
import pandas as pd

In [0]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None
    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [0]:
# Function for moving into next url
def get_next_url(soup):
  for i in soup.select('a'):
    if 'rel="next"' in str(i):
      i = str(i).split()[1]
      i = i.replace('href=','').replace('"','')
      i = 'https://www.argenprop.com'+i
      return i

In [0]:
# Function for getting each house url 
def get_urls(soup):  
    urls = []
    for i in str(soup.select('a')).splitlines():
      if 'class="card-block"' in i:
        url = i.split()[3]
        url = url.replace('href="','').replace('">','')
        url = 'https://www.argenprop.com'+url
        urls.append(url)
    return list(set(urls))

In [0]:
def get_row(url):
  html = simple_get(url)
  soup = BeautifulSoup(html)
  col = {}

  val =['Cant. Baños:',
        'Cant. Dormitorios:',
        'Cant. Plantas:',
        'Cant. Cocheras:',
        'Tipo de Unidad:',
        'Tipo de operación:',
        'Precio:',
        'Sup. Cubierta:',
        'Sup. Terreno:',
        'Antiguedad:',]

  for i in soup.get_text().splitlines():
    for v in val:
      if v in i:
        if v == 'Precio:':
          if 'USD' in i:
            i = re.findall("[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", i)
            i = ''.join(i)
            i = i.replace('.', '')
            col[v] = i
          elif '$' in i:
            i = re.findall("[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", i)
            i = ''.join(i)
            i = i.replace('.', '')
            ff = str(int(int(i)/38.8000))
            col[v] = ff
        elif v == 'Sup. Cubierta:' or v == 'Sup. Terreno:':
          col[v] = i.replace(v+' ','').replace(' m2','')
        else:
          col[v] = i.replace(v+' ','')

  for i in str(soup).splitlines():
    if 'property-titlebar-sub-header' in str(i):
      col['Barrio:'] = i.split(', ')[0].replace('<h2 class="property-titlebar-sub-header">','').replace('\'','').replace('Casa en Venta en ','')
      col['Ciudad:'] = i.split(', ')[1].replace('</h2>','')

  for i in soup.select('p'):
    if 'fa-map-marker' in str(i):
      i = str(i).split('</i> ')[1].replace('</p>','')
      col['Calle:'] = i 

  return col

In [0]:
# Final function for executing previous function and getting data 
def get_data(url, search):
  row = [] 
  try:
    for x in range(search):
      try:
        soup = BeautifulSoup(simple_get(url))
        urls = get_urls(soup)
      except:
        print('1: ',url)

      for i in urls:
        try:
          row.append(get_row(i))
        except:
          print('2: ', i)
      try:
        url = get_next_url(soup)
      except:
        print('3: ',url)
      print(x)   
    return row
  except:
    return row

In [7]:
# getting row data
url = 'https://www.argenprop.com/casa-venta-provincia-cordoba-arg'
x = get_data(url,140)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139


In [14]:
# Converting row data into pandas DataFrame
columns = ['Tipo de Unidad:',
           'Zona',
           'Ciudad:',
           'Barrio:',
           'Calle:',
           'Tipo de Barrio',
           'Apto Escritura', 
           'Apto Crédito',
           'Etapa de Construcción', 
           'Antiguedad:', 
           'Cant. Plantas:',
           'Cant. Dormitorios:',
           'Cant. Baños:',
           'Cant. Cocheras:',
           'Cobertura Cocheras:',
           'Sup. Terreno:',
           'Sup. Cubierta:',    
           'Precio:',]
        
dataFrame = pd.DataFrame(x, columns=columns)

dataFrame.columns = ['Tipo de Propiedad',
                     'Zona',
                     'Ciudad', 
                     'Barrio', 
                     'Calle', 
                     'Tipo de Barrio',
                     'Apto Escritura', 
                     'Apto Crédito',
                     'Etapa de Construcción', 
                     'Antigüedad', 
                     'Cantidad de Plantas',
                     'Cantidad de Dormitorios',
                     'Cantidad de Baños',
                     'Cantidad de Cocheras',
                     'Cobertura Cocheras',
                     'Superficie Total m2',
                     'Superficie Cubierta m2',
                     'Valor']

dataFrame.loc[dataFrame['Ciudad'] == 'Cordoba', 'Ciudad'] =  'Córdoba'
dataFrame

,Tipo de Propiedad,Zona,Ciudad,Barrio,Calle,Tipo de Barrio,Apto Escritura,Apto Crédito,Etapa de Construcción,Antigüedad,Cantidad de Plantas,Cantidad de Dormitorios,Cantidad de Baños,Cantidad de Cocheras,Cobertura Cocheras,Superficie Total m2,Superficie Cubierta m2,Valor
0,Casa,NaN,Córdoba,Liceo General Paz,NaN,NaN,NaN,NaN,NaN,20,2,3,3,2,NaN,300,230,150000
1,Casa,NaN,Córdoba,Bajo Palermo,Luis Galeano 1400,NaN,NaN,NaN,NaN,51,1,4,3,2,NaN,776,286,235000
2,Duplex,NaN,Córdoba,Villa Rivera Indarte,Los Reartes 8900,NaN,NaN,NaN,NaN,6,1,2,1,NaN,NaN,330,70,82000
3,Chalet,NaN,Pdo. de Punilla,Villa Carlos Paz,Los Cerros 1400,NaN,NaN,NaN,NaN,15,3,4,4,1,NaN,450,370,420000
4,Casa,NaN,Countries y Barrios Cerrados Cordoba Capital,Las Delicias,NaN,NaN,NaN,NaN,NaN,6,2,3,3,2,NaN,1.600,350,620000
5,Casa,NaN,Córdoba,Lomas de la Carolina,Country Lomas De La Carolina 0,NaN,NaN,NaN,NaN,13,2,4,5,3,NaN,1.500,580,830000
6,Casa,NaN,Córdoba,San Vicente,Agustin Garzon 3200,NaN,NaN,NaN,NaN,50,2,3,2,1,NaN,185,161,135000
7,NaN,NaN,Córdoba,Villa Belgrano,Gay Lussac 6200,NaN,NaN,NaN,NaN,25,NaN,4,3,NaN,NaN,1.714,446,480000
8,NaN,NaN,Córdoba,Lomas de la Carolina,Monseñor Pablo Cabrera 2000,NaN,NaN,NaN,NaN,10,NaN,4,2,NaN,NaN,302,210,170000
9,Casa,NaN,Córdoba,Urca,Mariano Larra 4000,NaN,NaN,NaN,NaN,23,2,3,4,2,NaN,600,248,320000


In [0]:
dataFrame.to_csv('Casas_venta_argenprop.csv', encoding='utf_8_sig')